# Import required modules

In [1]:
import pandas as pd
import pulp

# 1) Problem Statement

In [2]:
investment_options = pd.DataFrame({'Type':['Equity','SpecialEquity','Balanced','Foreign','Bond'],
                                   'Annual_Growth':[0.15,0.21,0.11,0.19,0.08],
                                   'Probability_Loss':[0.18,0.31,0.09,0.19,0.03]})
investment_options['Probability_Success'] = 1 - investment_options['Probability_Loss']
investment_options.sort_values('Type',inplace=True)
investment_options

,Type,Annual_Growth,Probability_Loss,Probability_Success
2,Balanced,0.11,0.09,0.91
4,Bond,0.08,0.03,0.97
0,Equity,0.15,0.18,0.82
3,Foreign,0.19,0.19,0.81
1,SpecialEquity,0.21,0.31,0.69


# 2) Variable names required for solver

In [5]:
per_ast_lower_bound = 0
var_names = []
for var in investment_options['Type']:
    var_names.append(pulp.LpVariable(var, lowBound=per_ast_lower_bound))
investment_options['var_name'] = var_names
investment_options.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 2 to 1
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Type                 5 non-null      object 
 1   Annual_Growth        5 non-null      float64
 2   Probability_Loss     5 non-null      float64
 3   Probability_Success  5 non-null      float64
 4   var_name             5 non-null      object 
dtypes: float64(3), object(2)
memory usage: 240.0+ bytes


# 3) Initializing the solver

In [3]:
maximize_rets = pulp.LpProblem("Maximizing_the_portfolio_returns", pulp.LpMaximize)
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
None
VARIABLES

# 4) Creating objective function

In [6]:
investment_options['objective_function'] = investment_options['var_name'] * (investment_options['Annual_Growth'] * investment_options['Probability_Success'])
portfolio_rets_eq = list(investment_options['objective_function'])
maximize_rets += pulp.lpSum([i for i in portfolio_rets_eq]) ,'Objective_(Portfolio_PL)'
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
0.10010000000000001*Balanced + 0.0776*Bond + 0.123*Equity + 0.1539*Foreign + 0.14489999999999997*SpecialEquity + 0.0
VARIABLES
Balanced Continuous
Bond Continuous
Equity Continuous
Foreign Continuous
SpecialEquity Continuous

# 5) Adding Constraints

## Sum of all weights to be equal to 1

In [7]:
sum_of_wts = list(investment_options['var_name'])
maximize_rets += pulp.lpSum([i for i in sum_of_wts]) == 1 ,'Sum_of_wts_=_1'
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
0.10010000000000001*Balanced + 0.0776*Bond + 0.123*Equity + 0.1539*Foreign + 0.14489999999999997*SpecialEquity + 0.0
SUBJECT TO
Sum_of_wts_=_1: Balanced + Bond + Equity + Foreign + SpecialEquity = 1

VARIABLES
Balanced Continuous
Bond Continuous
Equity Continuous
Foreign Continuous
SpecialEquity Continuous

## (Bonds + Balanced) >= 35%

In [ ]:
balanced_var = investment_options.loc[investment_options['Type']=='Balanced','var_name']
bond_var = investment_options.loc[investment_options['Type']=='Bond','var_name']
maximize_rets += pulp.lpSum([balanced_var,bond_var])>= 0.35, 'sum(Bnd,Bal)>=35%'
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
0.10010000000000001*Balanced + 0.0776*Bond + 0.123*Equity + 0.1539*Foreign + 0.14489999999999997*SpecialEquity + 0.0
SUBJECT TO
Sum_of_wts_=_1: Balanced + Bond + Equity + Foreign + SpecialEquity = 1

sum(Bnd,Bal)>=35%: Balanced + Bond >= 0.35

VARIABLES
Balanced Continuous
Bond Continuous
Equity Continuous
Foreign Continuous
SpecialEquity Continuous

## Equity >= 0.5 * (Equity + Special Equity + Foreign)

In [ ]:
equity_var = investment_options.loc[investment_options['Type']=='Equity','var_name']
special_eq_var = investment_options.loc[investment_options['Type']=='SpecialEquity','var_name']
foreign_var = investment_options.loc[investment_options['Type']=='Foreign','var_name']
maximize_rets += pulp.lpSum([equity_var,- special_eq_var,- foreign_var]) >= 0, 'Eq>=sum(SpEq,For)'
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
0.10010000000000001*Balanced + 0.0776*Bond + 0.123*Equity + 0.1539*Foreign + 0.14489999999999997*SpecialEquity + 0.0
SUBJECT TO
Sum_of_wts_=_1: Balanced + Bond + Equity + Foreign + SpecialEquity = 1

sum(Bnd,Bal)>=35%: Balanced + Bond >= 0.35

Eq>=sum(SpEq,For): Equity - Foreign - SpecialEquity >= 0

VARIABLES
Balanced Continuous
Bond Continuous
Equity Continuous
Foreign Continuous
SpecialEquity Continuous

## Expected loss percentage to be under 10%

In [ ]:
expected_loss = list(investment_options['var_name'] * investment_options['Probability_Loss'])
maximize_rets += pulp.lpSum([i for i in expected_loss]) <= 0.1 ,'Expected_Loss<=10%'
maximize_rets

Maximizing_the_portfolio_returns:
MAXIMIZE
0.10010000000000001*Balanced + 0.0776*Bond + 0.123*Equity + 0.1539*Foreign + 0.14489999999999997*SpecialEquity + 0.0
SUBJECT TO
Sum_of_wts_=_1: Balanced + Bond + Equity + Foreign + SpecialEquity = 1

sum(Bnd,Bal)>=35%: Balanced + Bond >= 0.35

Eq>=sum(SpEq,For): Equity - Foreign - SpecialEquity >= 0

Expected_Loss<=10%: 0.09 Balanced + 0.03 Bond + 0.18 Equity + 0.19 Foreign
 + 0.31 SpecialEquity <= 0.1

VARIABLES
Balanced Continuous
Bond Continuous
Equity Continuous
Foreign Continuous
SpecialEquity Continuous

# 6) Run the solver

In [ ]:
maximize_rets.solve()
print('Solver Status: ',pulp.LpStatus[maximize_rets.status])
print("The optimized solution :", pulp.value(maximize_rets.objective))

Solver Status:  Optimal
The optimized solution : 0.105080644965


# 7) Get the final weights

In [ ]:
final_wts = []
for variable in maximize_rets.variables():
    final_wts.append(variable.varValue)
    #print(variable.name, variable.varValue)
investment_options['final_wts'] = final_wts
investment_options

Balanced 0.0
Bond 0.5483871
Equity 0.22580645
Foreign 0.22580645
SpecialEquity 0.0


,Type,Annual_Growth,Probability_Loss,Probability_Success,var_name,objective_function,final_wts
2,Balanced,0.11,0.09,0.91,Balanced,{Balanced: 0.10010000000000001},0.000000
4,Bond,0.08,0.03,0.97,Bond,{Bond: 0.0776},0.548387
0,Equity,0.15,0.18,0.82,Equity,{Equity: 0.123},0.225806
3,Foreign,0.19,0.19,0.81,Foreign,{Foreign: 0.1539},0.225806
1,SpecialEquity,0.21,0.31,0.69,SpecialEquity,{SpecialEquity: 0.14489999999999997},0.000000


# 8) Sense check

In [ ]:
sum_of_all_wts = investment_options['final_wts'].sum()
print('Sum of all wts: ',sum_of_all_wts)

Sum of all wts:  1.0


In [ ]:
# (Balanced + Bond) > 0.35

In [ ]:
# (EQ>= 0.5 * (Eq+Sp_Eq+Foreign))

In [ ]:
prob_loss = (investment_options['final_wts'] * investment_options['Probability_Loss']).sum()
print('Probability of loss: ',prob_loss)

Probability of loss:  0.09999999949999999
